### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import fnmatch
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_report
from covid_model.utils import setup, get_filepath_prefix, db_engine
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221213_scenario_projections.ipynb")

2022/12/13 11:49:53 INFO|============================================================


In [3]:
# create an object called "engine" by running the db_engine function
engine = db_engine()

# get the date of the last reported hospitalization data point from EMResource (to set the stop date)
last_hosp = pd.read_sql('select max(measure_date) from emresource.hospitalized', con = engine).iloc[0,0].isoformat()

### Fit an initial scenario through the end of February 2022

In [12]:
# set inputs for the model

# first establish the model start date (use the isoformat command to turn it into a string)
startdate = dt.date(2020, 1, 24).isoformat()
# then get today's date (so we can set end dates automatically)
#today = dt.datetime.today().strftime("%Y-%m-%d")
# now calculate the end date (let's do 90 days from the date of the last reported hospitalization
# but it can really be as long we we want)
plus90 = dt.datetime.strptime(last_hosp, "%Y-%m-%d").date() + dt.timedelta(days = 90)
# turn into a string
enddate = plus90.isoformat()

# set model arguments
model_args = {
    'params_defs': 'covid_model/analysis/20221202_dec_cdphe_slides/20221202_scenario_params.json',
    #'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221202_dec_cdphe_slides/20221202_vacc_proj_params.json',
    'start_date': startdate,
    'end_date': enddate,
    'hosp_reporting_frac': {"2020-01-01": 1, "2022-03-01": 0.932, "2022-03-15": 0.868, "2022-03-30": 0.8},
    'lt5_vacc_adjust': {"param": "immunity", "attrs": {'age': '0-19', 'vacc': 'shot1'},
                          "mults": {"2020-01-01": 1,
                                    "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                    "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                    "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                    "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                    "2022-08-10": 0.95 + 0.05*vacc_eff_lt5,
                                    "2022-08-30": 0.94 + 0.06*vacc_eff_lt5,
                                    "2022-09-20": 0.93 + 0.07*vacc_eff_lt5},
                        "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}
}

# set output directory and end date for fit (i.e., the last day of the hospitalization data)
fit_args = {
    'outdir': outdir,
    'fit_end_date': last_hosp
}

In [13]:
# if you want to refit the model, run this code
# this creates an object called "model" by passing the fit_args and model_args through the do_single_fit function
# otherwise, run the next cell down
model = do_single_fit(**fit_args, **model_args)

2022/12/13 12:16:09 INFO|-------------------------|{} Retrieving vaccinations data
2022/12/13 12:16:11 INFO|-------------------------|{} Constructing vaccination projections
2022/12/13 12:16:12 INFO|-------------------------|{} Retrieving hospitalizations data
2022/12/13 12:16:13 INFO|----------------------|{} Prepping Model
2022/12/13 12:16:13 INFO|-----------------------|{} Prepping Model
2022/12/13 12:17:30 INFO|----------------------|{} Model prepped for fitting in 76.53035287500006 seconds.
2022/12/13 12:17:30 INFO|----------------------|{} Will fit 35 times
2022/12/13 12:19:17 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/35 completed in 107.50872154099989 seconds: {0: {'co': 0.8306366938274515}, 14: {'co': 0.6979623977021482}, 28: {'co': 0.5697825887250856}, 42: {'co': 0.0028644801015151513}, 56: {'co': 0.681466989081648}, 70: {'co': 0.8912779910149108}}
2022/12/13 12:19:17 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/12/

2022/12/13 12:24:32 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/35 completed in 9.030785292000019 seconds: {504: {'co': 0.7927614869377158}, 518: {'co': 0.786297855789007}, 532: {'co': 0.7661786116173283}, 546: {'co': 0.7176009769087839}, 560: {'co': 0.7506313143440976}, 574: {'co': 0.7457868870597185}}
2022/12/13 12:24:32 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/12/13 12:24:47 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/35 completed in 13.345455958999992 seconds: {532: {'co': 0.7655333903795535}, 546: {'co': 0.7194148615933602}, 560: {'co': 0.7472635843985098}, 574: {'co': 0.7558547275972319}, 588: {'co': 0.7926874527566158}, 602: {'co': 0.7730462157092053}}
2022/12/13 12:24:47 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/12/13 12:25:03 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/35 completed in 14.60507266600007 seconds: {5

2022/12/13 12:29:47 INFO|-----------------------|{'fit_batch': '34', 'run_type': 'fit'}: Running forward sim
2022/12/13 12:29:49 INFO|----------------------|{'fit_batch': '34', 'run_type': 'fit'}: Uploading final results
2022/12/13 12:29:54 INFO|----------------------|{'fit_batch': '34', 'run_type': 'fit'}: spec_id: 4561


In [ ]:
# if you want to pass an old fit spec ID through, run this code
# otherwise, refit using the code above
#model_args = {
    #'base_spec_id': 4515, # use the spec id that was output from the model fit you want to pass through
#}
# create an object called "model" by passing the above-mentioned model_args through CovidModel
#model = CovidModel(**model_args)
# prep the model
#model.prep()

In [14]:
# output the TC json file to csv

# path to model output where tc .json's are
results_dir = "../output/20221213_scenario_projections.ipynb"
tc_files = []
pattern = '*run_type_fit__model_tc.json'

# grab all forecast tc json files
tc_files = [os.path.join(outdir, file) for file in os.listdir(outdir) if fnmatch.fnmatch(file, pattern)]

# turn into .csv files and put in output directory
for file in tc_files:
    df = pd.read_json(file)
    file_new = file.replace('.json','.csv' )
    df.to_csv(file_new)

In [ ]:
# Brittney, can you write a small script that pulls the last few TC values (say eight weeks back)
# then extract the min, max, and mean of that range?

### Run the model

In [15]:
# now create the reports for the model, indexing through all the values of TC we want to simulate
logging.info('Projecting')
# input the three TC values: min, mean, and max
for tc in [0.7177, 0.7708, 0.7964]:
    logging.info('')
    model.update_tc({model.date_to_t(last_hosp): {'co': tc}}, replace = False)
    model.tags.update({'tc': tc})
    model.solve_seir()
    do_create_report(model, outdir=outdir, prep_model=False, solve_model=False, immun_variants=['ba45', 'emv'], from_date='2022-01-01')

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), enddate)
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), enddate)
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

    logging.info('Running reports')

2022/12/13 12:31:09 INFO|Projecting
2022/12/13 12:31:09 INFO|
2022/12/13 12:31:13 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/13 12:33:09 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.7177}: Running forward sim
2022/12/13 12:33:10 INFO|Running reports
2022/12/13 12:33:10 INFO|
2022/12/13 12:33:14 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/13 12:35:09 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.7708}: Running forward sim
2022/12/13 12:35:10 INFO|Running reports
2022/12/13 12:35:10 INFO|
2022/12/13 12:35:15 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/12/13 12:37:09 INFO|{'fit_batch': '34', 'run_t